In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm # для прогрес-бару

# --- 0. Налаштування та визначення пристрою ---
print("--- 0. Setup and Device Configuration ---")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# --- 1. Завантаження та очищення даних ---
print("\n--- 1. Data Loading and Cleaning ---")
# train_df_raw = pd.read_csv('../data/liar/train_filtered.csv')
# val_df_raw = pd.read_csv('../data/liar/valid_filtered.csv')
# test_df_raw = pd.read_csv('../data/liar/test_filtered.csv')

train_df_raw = pd.read_csv('../data/fakenewsnet_dataset/combined_train.csv')
val_df_raw = pd.read_csv('../data/fakenewsnet_dataset/combined_val.csv')
test_df_raw = pd.read_csv('../data/fakenewsnet_dataset/combined_test.csv')

In [ ]:

def clean_dataframe(df, text_column='statement', label_column='binary_label'):
    print(f"Initial shape for {df.name if hasattr(df, 'name') else 'DataFrame'}: {df.shape}")
    df.dropna(subset=[text_column], inplace=True)
    df = df[df[text_column].apply(lambda x: isinstance(x, str) and x.strip() != '')].copy() # Додано .copy() для уникнення SettingWithCopyWarning
    print(f"Shape after cleaning: {df.shape}")
    return df

train_df_raw.name = 'Train_Raw' # Для логування
val_df_raw.name = 'Val_Raw'
test_df_raw.name = 'Test_Raw'

train_df = clean_dataframe(train_df_raw)
val_df = clean_dataframe(val_df_raw)
test_df = clean_dataframe(test_df_raw)

In [ ]:

# --- 2. Ініціалізація токенізатора BERT ---
print("\n--- 2. BERT Tokenizer Initialization ---")
MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# --- 3. Створення класу Dataset ---
print("\n--- 3. Custom Dataset Class ---")
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.texts = texts.tolist() # Перетворюємо на список для індексації
        self.labels = labels.tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True, # Додає [CLS] та [SEP]
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_attention_mask=True,
            return_tensors='pt'  # Повертає PyTorch тензори
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

MAX_LEN = 256 # Максимальна довжина послідовності для BERT
train_dataset = NewsDataset(train_df['statement'], train_df['binary_label'], tokenizer, MAX_LEN)
val_dataset = NewsDataset(val_df['statement'], val_df['binary_label'], tokenizer, MAX_LEN)
test_dataset = NewsDataset(test_df['statement'], test_df['binary_label'], tokenizer, MAX_LEN)


In [ ]:

# --- 4. Створення DataLoader'ів ---
print("\n--- 4. DataLoaders Creation ---")
BATCH_SIZE = 16 # Розмір батчу
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# --- 5. Завантаження моделі BERT ---
print("\n--- 5. BERT Model Loading ---")
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2) # 2 класи: фейк/правда
model = model.to(device)

In [ ]:

print("\n--- 6. Optimizer and Scheduler Setup ---")
LEARNING_RATE = 2e-5
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

NUM_EPOCHS = 5 # Кількість епох
num_training_steps = NUM_EPOCHS * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", # Тип планувальника
    optimizer=optimizer,
    num_warmup_steps=0, # Кількість кроків для "розігріву"
    num_training_steps=num_training_steps
)

# --- 7. Функція для оцінки на валідаційній вибірці ---
def evaluate_model_on_val(model, data_loader, device):
    model.eval()
    total_val_loss = 0
    correct_predictions = 0
    total_predictions = 0
    all_val_labels = []
    all_val_preds = []

    with torch.no_grad():
        for batch in data_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            total_val_loss += loss.item()

            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            labels = batch['labels']

            correct_predictions += torch.sum(preds == labels)
            total_predictions += labels.size(0)

            all_val_labels.extend(labels.cpu().numpy())
            all_val_preds.extend(preds.cpu().numpy())

    avg_val_loss = total_val_loss / len(data_loader)
    val_accuracy = correct_predictions.double() / total_predictions

    # Розрахунок F1 для валідації
    val_f1 = classification_report(all_val_labels, all_val_preds, output_dict=True, zero_division=0)['weighted avg']['f1-score']

    return avg_val_loss, val_accuracy.item(), val_f1


In [ ]:

# --- 8. Цикл навчання моделі ---
print("\n--- 8. Model Training Loop ---")
best_val_f1 = -1 # Для збереження найкращої моделі за F1 на валідації
history_bert = {'train_loss': [], 'val_loss': [], 'train_accuracy': [], 'val_accuracy': [], 'val_f1': []}

for epoch in range(NUM_EPOCHS):
    model.train()
    total_train_loss = 0
    correct_train_predictions = 0
    total_train_samples = 0

    loop = tqdm(train_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS}', leave=True)
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        total_train_loss += loss.item()

        # Розрахунок точності на тренувальному батчі (опціонально, для моніторингу)
        logits_train = outputs.logits
        preds_train = torch.argmax(logits_train, dim=1)
        correct_train_predictions += torch.sum(preds_train == batch['labels'])
        total_train_samples += batch['labels'].size(0)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Градієнтний кліппінг
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        loop.set_postfix(loss=loss.item())

    avg_train_loss = total_train_loss / len(train_loader)
    train_accuracy = correct_train_predictions.double() / total_train_samples
    history_bert['train_loss'].append(avg_train_loss)
    history_bert['train_accuracy'].append(train_accuracy.item())

    print(f"\nEpoch {epoch+1} - Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

    # Оцінка на валідаційній вибірці
    avg_val_loss, val_accuracy, val_f1 = evaluate_model_on_val(model, val_loader, device)
    history_bert['val_loss'].append(avg_val_loss)
    history_bert['val_accuracy'].append(val_accuracy)
    history_bert['val_f1'].append(val_f1)
    print(f"Epoch {epoch+1} - Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1: {val_f1:.4f}")

    # Збереження найкращої моделі
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        # Зберігаємо модель (рекомендований спосіб для Hugging Face)
        model.save_pretrained('./bert_classifier_liar_best_model/')
        tokenizer.save_pretrained('./bert_classifier_liar_best_model/') # Зберігаємо і токенізатор
        print(f"Epoch {epoch+1}: Best model saved with Val F1: {val_f1:.4f}")

print("Training finished.")

In [ ]:

# Завантаження найкращої моделі для фінальної оцінки
print("\nLoading best model for final evaluation...")
model = BertForSequenceClassification.from_pretrained('./bert_classifier_liar_best_model/')
model = model.to(device)


# --- 9. Графіки історії навчання ---
print("\n--- 9. Plotting Training History ---")
def plot_bert_history(history):
    plt.figure(figsize=(18, 5))

    plt.subplot(1, 3, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('BERT Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 3, 2)
    plt.plot(history['train_accuracy'], label='Train Accuracy')
    plt.plot(history['val_accuracy'], label='Validation Accuracy')
    plt.title('BERT Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 3, 3)
    plt.plot(history['val_f1'], label='Validation F1-Score')
    plt.title('BERT Model Validation F1-Score')
    plt.xlabel('Epoch')
    plt.ylabel('F1-Score')
    plt.legend()

    plt.tight_layout()
    plt.show()



plot_bert_history(history_bert)

history_df = pd.DataFrame({
    'Epoch': range(1, len(history_bert['train_accuracy']) + 1),
    'Train Accuracy': history_bert['train_accuracy'],
    'Validation Accuracy': history_bert['val_accuracy'],
    'Train Loss': history_bert['train_loss'],
    'Validation Loss': history_bert['val_loss'],
    'Validation F1': history_bert['val_f1']
})

# Виведення таблиці
print(history_df)


In [ ]:

# --- 10. Оцінка моделі на тестовій вибірці ---
print("\n--- 10. Final Evaluation on Test Set ---")
model.eval() # Переводимо модель в режим оцінки
all_test_preds = []
all_test_probs = []
all_test_labels = []

with torch.no_grad(): # Відключаємо обчислення градієнтів
    for batch in tqdm(test_loader, desc="Evaluating on Test Set"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)

        # Отримуємо ймовірності для позитивного класу
        probs = torch.softmax(outputs.logits, dim=1)[:, 1].cpu().numpy()
        # Отримуємо прогнозовані класи
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        labels = batch['labels'].cpu().numpy()

        all_test_probs.extend(probs)
        all_test_preds.extend(preds)
        all_test_labels.extend(labels)

print("\nClassification Report (Test Set):")
print(classification_report(all_test_labels, all_test_preds, zero_division=0))

# Матриця помилок
cm_test_bert = confusion_matrix(all_test_labels, all_test_preds)
plt.figure(figsize=(6,5))
sns.heatmap(cm_test_bert, annot=True, fmt='d', cmap='Blues')
plt.title('BERT Confusion Matrix (Test Set)')
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.show()

# ROC-крива
fpr_test_bert, tpr_test_bert, _ = roc_curve(all_test_labels, all_test_probs)
roc_auc_test_bert = auc(fpr_test_bert, tpr_test_bert)

plt.figure(figsize=(7,6))
plt.plot(fpr_test_bert, tpr_test_bert, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc_test_bert:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('BERT ROC Curve (Test Set)')
plt.legend(loc="lower right")
plt.show()

print(f"BERT Model Test AUC: {roc_auc_test_bert:.4f}")